In [3]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math
from embfts.util.DataSetUtil import DataSetUtil


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
data_set_util = DataSetUtil()

In [5]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/AirQualityUCI.csv', sep=';', decimal=',')
data = df.drop(labels=['Date','Time','Unnamed: 15', 'Unnamed: 16'], axis=1)
data.dropna(inplace=True)
#data = clean_dataset(data)
data.head()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


In [6]:
data.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,-34.207524,1048.990061,-159.090093,1.865683,894.595276,168.616971,794.990168,58.148873,1391.479641,975.072032,9.778305,39.485380,-6.837604
std,77.657170,329.832710,139.789093,41.380206,342.333252,257.433866,321.993552,126.940455,467.210125,456.938184,43.203623,51.216145,38.976670
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,0.600000,921.000000,-200.000000,4.000000,711.000000,50.000000,637.000000,53.000000,1185.000000,700.000000,10.900000,34.100000,0.692300
50%,1.500000,1053.000000,-200.000000,7.900000,895.000000,141.000000,794.000000,96.000000,1446.000000,942.000000,17.200000,48.600000,0.976800
75%,2.600000,1221.000000,-200.000000,13.600000,1105.000000,284.000000,960.000000,133.000000,1662.000000,1255.000000,24.100000,61.900000,1.296200
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000,44.600000,88.700000,2.231000


In [7]:
# frame as supervised learning
reframed = data_set_util.series_to_supervised_mimo(data, 1, 1)
reframed.head()

,CO(GT)(t-1),PT08.S1(CO)(t-1),NMHC(GT)(t-1),C6H6(GT)(t-1),PT08.S2(NMHC)(t-1),NOx(GT)(t-1),PT08.S3(NOx)(t-1),NO2(GT)(t-1),PT08.S4(NO2)(t-1),PT08.S5(O3)(t-1),...,C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
1,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,...,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,...,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,...,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,...,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888
5,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,...,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848


In [8]:
cols = reframed.columns
cols

Index(['CO(GT)(t-1)', 'PT08.S1(CO)(t-1)', 'NMHC(GT)(t-1)', 'C6H6(GT)(t-1)',
       'PT08.S2(NMHC)(t-1)', 'NOx(GT)(t-1)', 'PT08.S3(NOx)(t-1)',
       'NO2(GT)(t-1)', 'PT08.S4(NO2)(t-1)', 'PT08.S5(O3)(t-1)', 'T(t-1)',
       'RH(t-1)', 'AH(t-1)', 'CO(GT)(t)', 'PT08.S1(CO)(t)', 'NMHC(GT)(t)',
       'C6H6(GT)(t)', 'PT08.S2(NMHC)(t)', 'NOx(GT)(t)', 'PT08.S3(NOx)(t)',
       'NO2(GT)(t)', 'PT08.S4(NO2)(t)', 'PT08.S5(O3)(t)', 'T(t)', 'RH(t)',
       'AH(t)'],
      dtype='object')

In [9]:
df_forecats_columns = ['CO(GT)(t)', 'PT08.S1(CO)(t)', 'NMHC(GT)(t)',
       'C6H6(GT)(t)', 'PT08.S2(NMHC)(t)', 'NOx(GT)(t)', 'PT08.S3(NOx)(t)',
       'NO2(GT)(t)', 'PT08.S4(NO2)(t)', 'PT08.S5(O3)(t)', 'T(t)', 'RH(t)',
       'AH(t)']

In [10]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "variable":[]
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,'CO(GT)(t-1)':'AH(t-1)']
            df_test = ttest.loc[:,'CO(GT)(t-1)':'AH(t-1)']
            df_original = ttest.loc[:,'CO(GT)(t)':'AH(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
                original = original[:len(original)-1]
                forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = mean_absolute_error(original,forecast)
                r2 = r2_score(original,forecast)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = Measures.rmse(original,forecast)
                mape = Measures.mape(original,forecast)
                
                #nrmse = cal_nrmse(rmse, original)

                result["rmse"].append(rmse)
                #result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

## Sliding Window: MIMO KPCA-NSFTS

In [11]:
num_components_pca = 4
order_fts_model = 1
npart = 50
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [12]:
kpca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 311
--------------------
training window 622
--------------------
training window 933
--------------------
training window 1244
--------------------
training window 1555
--------------------
training window 1866
--------------------
training window 2177
--------------------
training window 2488
--------------------
training window 2799
--------------------
training window 3110
--------------------
training window 3421
--------------------
training window 3732
--------------------
training window 4043
--------------------
training window 4354
--------------------
training window 4665
--------------------
training window 4976
--------------------
training window 5287
--------------------
training window 5598
--------------------
training window 5909
--------------------
training window 6220
--------------------
training window 6531
--------------------
training window 6842
--------------------
training window 715

In [13]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": []
}

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [14]:
print("Statistics MIMO KPCA-NSFTS(test): ")
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2
0,CO(GT)(t),5.567,4.508,203.646,0.988
1,PT08.S1(CO)(t),38.202,30.597,4.783,0.968
2,NMHC(GT)(t),8.221,6.651,3.334,0.994
3,C6H6(GT)(t),3.102,2.509,34.656,0.971
4,PT08.S2(NMHC)(t),46.178,36.863,5.911,0.968
5,NOx(GT)(t),27.070,21.717,13.308,0.971
6,PT08.S3(NOx)(t),41.061,32.642,5.649,0.968
7,NO2(GT)(t),13.529,10.857,9.379,0.975
8,PT08.S4(NO2)(t),47.552,38.105,5.313,0.970
9,PT08.S5(O3)(t),60.246,48.054,7.264,0.966


## Sliding Window: MIMO PCA-NSFTS

In [17]:
num_components_pca = 4
order_fts_model = 1
npart = 50
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [18]:
pca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0


AttributeError: 'KernelPCA' object has no attribute 'X_transformed_fit_'

In [80]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": []
}

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2
0,CO(GT)(t),22.968,14.102,918.505,0.575
1,PT08.S1(CO)(t),64.900,45.510,5.813,0.887
2,NMHC(GT)(t),13.694,10.045,5.391,0.983
3,C6H6(GT)(t),5.069,3.311,45.647,0.884
4,PT08.S2(NMHC)(t),83.262,57.962,8.479,0.892
5,NOx(GT)(t),64.408,44.754,30.005,0.829
6,PT08.S3(NOx)(t),83.514,60.377,10.522,0.869
7,NO2(GT)(t),39.442,26.246,25.098,0.737
8,PT08.S4(NO2)(t),88.588,60.131,6.524,0.876
9,PT08.S5(O3)(t),121.823,88.057,12.595,0.861
